### Tests for Applying Gains Matrix

In [2]:
import cupy as cp
import numpy as np
from utils import *
from zp_puregpu_funcs_py import *
from invcov import *

hera_cal is not installed. Some gridding features unavailable.
pyfof is not installed. Some gridding features unavailable.


In [3]:
apply_gains_to_mat?
# undo_zeropad?

Signature:
apply_gains_to_mat(
    gains: cupy.ndarray,
    mat: cupy.ndarray,
    edges: cupy.ndarray,
    ant_1_array: cupy.ndarray,
    ant_2_array: cupy.ndarray,
    xp: Any,
    is_zeropadded: bool = True,
)
Docstring:
Apply a pair of complex gains to a matrix. Utilizes the Re/Im split.
Only accounts for "one half" of the gain application, meaning the 
function is really performing eg. (g_1g_2*\Delta_{1,2}), where it is 
understood that antenna's 1 and 2 belong to the baseline sitting at
the same row as that baseline row in the \Delta (\Sigma) matrix. Note that 
although the matrix provided may be zeropadded, the gain matrix is zeropadded 
here and as such, should always be provided in an un-zeropadded (original) format.

NOTE: Could be smart in the future to wrap the zeropadding of the gain 
    matrix in a separate function.

Params
------
gains
    1D array of Re/Im alternating gains to be applied to the source or
    diffuse matrices. Contains one set of Re/Im gains for all an

In [4]:
summarize_benchmark_results?

Signature: summarize_benchmark_results(function, *args)
Docstring:
Use the CuPy benchmark function and nicely print out just the
total time spent on both the gpu and the cpu.
File:      ~/corrcal_gpu_pipeline/pipeline/utils.py
Type:      function

In [5]:
#simulation params relevant for testing application of gains to a matrix
# n_ant = 15
n_bl = 
n_gains = 4*n_ant
n_eig = 3
xp = cp  #run things on the gpu using cupy

#this might be the easiest (and most general) way to devise an edges
# array, though we hard code an ex. edges array to be sure it fits
# the desired format of having no odd entries.
edges = (xp.unique(xp.random.randint(1, n_bl/2-1, size = 3)*2))
edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
print(f"The edges of the redundant blocks have indices{edges}")

#some random noise, diffuse, and source covariance matrices
xp = cp
sim_diff_mat = xp.random.rand(n_bl, n_eig, dtype = 'float64')
sim_gains = cp.random.rand(n_gains, dtype = 'float64') #Re/Im split + ant1 & ant 2 = 4*n_ant
ant_1_array = cp.arange(n_ant)
ant_2_array = cp.arange(n_ant, 2*n_ant)

#zeropad the noise, diff, source mats
zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)

#Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)

The edges of the redundant blocks have indices[ 0  4  8 14 30]


In [6]:
applied_gains = apply_gains_to_mat(sim_gains, sim_diff_mat_3d, edges, ant_1_array, ant_1_array, cp, True)

In [7]:
print(f"Input mat has shape {sim_diff_mat_3d.shape}")
print()
print(f"Gains applied mat has shape {applied_gains.shape}")
print()
print(f"The mat with gains applied looks like:\n\n{applied_gains}")

Input mat has shape (4, 16, 3)

Gains applied mat has shape (4, 16, 3)

The mat with gains applied looks like:

[[[4.30253351e-02 4.61695153e-02 5.35443851e-02]
  [4.95765444e-02 1.69144867e-02 5.72796261e-02]
  [5.06890765e-01 1.95219456e-01 4.12420875e-01]
  [4.11393123e-03 5.34941703e-01 4.66031133e-01]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]]

 [[1.84190039e+00 4.66043203e-01 3.78107459e-02]
  [5.97790334e-01 9.33815176e-01 5.38353165e-01]
  [9

#### Random Debugging tests

In [8]:
m = np.random.rand(2, 5, 3)
g = np.random.rand(10)

gs = g[:, None] * g[:, None]
gs = gs.reshape(m.shape[0], m.shape[1], 1)

print(gs.shape)
print(m.shape)
print(g.shape)
print(m)
print(m[:, ::2])
out = gs[:, ::2] * m[:, ::2]

(2, 5, 1)
(2, 5, 3)
(10,)
[[[0.21939056 0.64113813 0.46214428]
  [0.62058711 0.4690525  0.58296433]
  [0.12203577 0.90374314 0.14956712]
  [0.03301137 0.5971557  0.09848793]
  [0.99692167 0.2562628  0.27176516]]

 [[0.97666838 0.91647957 0.57702342]
  [0.33962314 0.16453449 0.3320102 ]
  [0.3976622  0.69911254 0.73140774]
  [0.20320159 0.98686286 0.48927323]
  [0.64270555 0.57394148 0.32598416]]]
[[[0.21939056 0.64113813 0.46214428]
  [0.12203577 0.90374314 0.14956712]
  [0.99692167 0.2562628  0.27176516]]

 [[0.97666838 0.91647957 0.57702342]
  [0.3976622  0.69911254 0.73140774]
  [0.64270555 0.57394148 0.32598416]]]


In [9]:
a = np.arange(5)
b = np.arange(5,10)
c = np.random.rand((a.shape[0] + b.shape[0]))
print(c)
d = c[a]
e = c[b]

print(d)
print()
print(e)
print()
print(np.concatenate((d, e)))
print()
aa = np.concatenate((d, e))

print(aa[:int(len(aa)/2)])

print(aa[int(len(aa)/2):])

[0.23682222 0.86665771 0.02597364 0.81493913 0.94997188 0.54719539
 0.7356526  0.66542159 0.00969051 0.8292626 ]
[0.23682222 0.86665771 0.02597364 0.81493913 0.94997188]

[0.54719539 0.7356526  0.66542159 0.00969051 0.8292626 ]

[0.23682222 0.86665771 0.02597364 0.81493913 0.94997188 0.54719539
 0.7356526  0.66542159 0.00969051 0.8292626 ]

[0.23682222 0.86665771 0.02597364 0.81493913 0.94997188]
[0.54719539 0.7356526  0.66542159 0.00969051 0.8292626 ]


#### Check ZP computes the same thing as corrcal

In [10]:
#Current corrcal gain application function
def apply_gains_to_mat_corrcal(gains, mat, ant_1_array, ant_2_array):
    """Apply a gain-like matrix to a provided matrix."""
    complex_gains = gains[::2] + 1j*gains[1::2]
    gain_mat = (
        complex_gains[ant_1_array,None] * complex_gains[ant_2_array,None].conj()
    )
    out = np.zeros_like(mat)
    out[::2] = gain_mat.real * mat[::2] - gain_mat.imag * mat[1::2]
    out[1::2] = gain_mat.imag * mat[::2] + gain_mat.real * mat[1::2]
    return out

In [11]:
#simulation params relevant for testing application of gains to a matrix
n_ant = 15
n_bl = 2*n_ant
n_gains = 4*n_ant
n_eig = 3
xp = cp  #run things on the gpu using cupy

#this might be the easiest (and most general) way to devise an edges
# array, though we hard code an ex. edges array to be sure it fits
# the desired format of having no odd entries.
edges = (xp.unique(xp.random.randint(1, n_bl/2-1, size = 3)*2))
edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
print(f"The edges of the redundant blocks have indices{edges}")

#some random noise, diffuse, and source covariance matrices
xp = cp
sim_diff_mat = xp.random.rand(n_bl, n_eig, dtype = 'float64')
sim_gains = cp.random.rand(n_gains, dtype = 'float64') #Re/Im split + ant1 & ant 2 = 4*n_ant
ant_1_array = cp.arange(n_ant)
ant_2_array = cp.arange(n_ant, 2*n_ant)

#zeropad the noise, diff, source mats
zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)

#Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)


The edges of the redundant blocks have indices[ 0  2  8 10 30]


In [12]:
# out_corrcal = apply_gains_to_mat_corrcal(sim_gains, sim_diff_mat, ant_1_array, ant_2_array)
out_gpu = apply_gains_to_mat(sim_gains, sim_diff_mat_3d, edges, ant_1_array, ant_2_array, cp)
out_gpu_corrcal = apply_gains_to_mat(sim_gains, sim_diff_mat, edges, ant_1_array, ant_2_array, cp, False)
out_gpu_resh = undo_zeropad(out_gpu, edges, cp)

In [13]:
#should return True if my routine computes the same thing as the current corrcal routine
print(cp.allclose(out_gpu_resh, out_gpu_corrcal))

True


#### Timing test for np vs cp versions

In [43]:
#simulation params relevant for testing application of gains to a matrix
n_bl = 240000  # something like 2*N_ant^2 for Re/Im split
n_ant = 512
n_gains = 2*n_bl  #one for each antenna and one for each Re and Im part
n_eig = 3
xp = cp  #run things on the gpu using cupy

#this might be the easiest (and most general) way to devise an edges
# array, though we hard code an ex. edges array to be sure it fits
# the desired format of having no odd entries.
if xp == cp:

    edges = (xp.unique(xp.random.randint(1, n_bl/2-1, size = n_ant)*2))
    edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
    print(f"The edges of the redundant blocks have indices{edges}")

    #some random noise, diffuse, and source covariance matrices
    sim_diff_mat = xp.random.rand(n_bl, n_eig, dtype = 'float64')
    sim_gains = cp.random.rand(n_gains, dtype = 'float64') #Re/Im split + ant1 & ant 2 = 4*n_ant
    
    #leave as n_bl/2 since indexex into cplex gains which are not Re/Im split
    ant_1_array = cp.arange(n_bl//2) 
    ant_2_array = cp.arange(n_bl//2)

    #zeropad the noise, diff, source mats
    zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)

    #Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
    sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)

elif xp == np:
    edges = (xp.unique(xp.random.randint(1, n_bl/2-1, size = n_ant)*2))
    edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
    print(f"The edges of the redundant blocks have indices{edges}")

    #some random noise, diffuse, and source covariance matrices
    sim_diff_mat = xp.random.rand(n_bl, n_eig).astype('float64')
    sim_gains = xp.random.rand(n_gains).astype('float64') #Re/Im split + ant1 & ant 2 = 4*n_ant
    ant_1_array = xp.arange(n_bl//2)
    ant_2_array = xp.arange(n_bl//2)

    #zeropad the noise, diff, source mats
    # zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)

    #Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
    # sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)

print(len(edges))

The edges of the redundant blocks have indices[     0    450    582    840   1894   2002   2378   2414   2992   5950
   6110   6524   7130   7172   7564   7956   8364   9218  10098  10322
  10746  10788  10864  11148  11538  12016  12066  12562  12930  14366
  14976  15266  15440  15868  17198  18384  18606  18962  19182  20220
  20422  20484  20568  20970  21710  22080  22152  22806  22998  24198
  24748  25418  25642  27326  27514  27612  28662  28688  28836  28930
  28992  29304  29446  29536  29698  30366  30586  30624  31568  32222
  32284  32496  33708  33980  34880  35488  35502  35796  36916  37332
  37662  37814  38192  38694  39190  39684  39690  40364  41266  41456
  42080  42552  44400  44518  45404  46074  46088  46390  46824  48336
  48904  50238  50552  50920  51522  52146  53342  53394  53566  53988
  54504  55294  55690  55974  56744  56798  57696  58672  58742  59156
  59424  59554  59882  60070  60260  61202  61286  63004  64116  64264
  64626  65050  65674  66336  6

In [42]:
print(type(sim_diff_mat))
print(type(sim_gains))
summarize_benchmark_results(apply_gains_to_mat, sim_gains, sim_diff_mat, edges, ant_1_array, ant_2_array, xp, False)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Time on cpu: 0.014120s
Time on gpu: 0.014666s


In [45]:
print(type(sim_diff_mat))
print(type(sim_gains))
summarize_benchmark_results(apply_gains_to_mat, sim_gains, sim_diff_mat_3d, edges, ant_1_array, ant_2_array, xp, True)

<class 'cupy.ndarray'>
<class 'cupy.ndarray'>
Time on cpu: 0.002538s
Time on gpu: 0.004685s
